# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [ ]:
import sqlite3    # Python interface for SQLite databases

In [ ]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [ ]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

From this point we will use __%%sql__ magic

In [ ]:
%load_ext sql
%sql sqlite:///wine.db

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [ ]:
%%sql SELECT *
FROM Master1
LIMIT 10;

* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [ ]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

![alt text](master1.png "Normalisation")

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

![alt text](master2.1.png "Normalisation")

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [ ]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

In [ ]:
print("\nContent of the database")
printSchema(conn)

In [ ]:
%%sql
SELECT *
FROM dummy;

Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [ ]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

Notice that we use `NULL` given that `None` is only used for display.

In [ ]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

#### Exercise 1.3

Create the new tables from Master1:

In [ ]:
%%sql DROP TABLE IF EXISTS wine;

-- Create wine table
CREATE TABLE wine AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1;
SELECT * FROM wine limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS producer;

-- Create producer table
CREATE TABLE producer AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1;
SELECT * FROM producer limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS production;

-- Create production table
CREATE TABLE production AS
SELECT DISTINCT NV, QTE, NP
FROM MASTER1
WHERE QTE is not NULL;
SELECT * FROM production limit 4;

#### Exercise 1.4

Create the new tables from Master2:

In [ ]:
%%sql DROP TABLE IF EXISTS wine;

-- Create wine table
CREATE TABLE wine AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2;
SELECT * FROM wine limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS purchase;

-- Create purchase table
CREATE TABLE purchase AS
SELECT DISTINCT NV, DATES, LIEU, QTE, NB
FROM MASTER2;
SELECT * FROM purchase limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS client;

-- Create client table
CREATE TABLE client AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2;
SELECT * FROM client limit 4;

In [ ]:
%%sql DROP TABLE IF EXISTS location;

-- Create location table
CREATE TABLE location AS
SELECT DISTINCT LIEU, REGION
FROM MASTER2;
SELECT * FROM location limit 4;

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [ ]:
%%sql
SELECT DISTINCT TYPE as type_of_buveurs
FROM client
WHERE TYPE is not null;

#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [ ]:
%%sql
SELECT wine.CRU, producer.REGION
FROM wine, producer, production
WHERE wine.NV=production.NV and production.NP=producer.NP
and (CRU="Pommard" or CRU="Brouilly");

#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [ ]:
%%sql
SELECT DISTINCT REGION
FROM (SELECT wine.CRU, producer.REGION
      FROM wine, producer, production
      WHERE wine.NV=production.NV and production.NP=producer.NP
      and (CRU="Pommard" or CRU="Brouilly")
      GROUP BY CRU);

In [ ]:
%%sql
SELECT DISTINCT REGION
FROM Wine, Producer, Production
WHERE Production.NV = Wine.NV AND Production.NP = Producer.NP
AND (CRU = "Pommard" or CRU = "Brouilly")
GROUP BY REGION
HAVING COUNT(REGION) = 2;

#### Exercise 2.4

Get the number of wines bought by CRU and Millésime

In [ ]:
%%sql
SELECT wine.CRU, wine.MILL, sum(purchase.QTE) as Number_of_wines_bought 
FROM wine, purchase
WHERE CRU is not null and MILL is not null and wine.NV=purchase.NV and QTE is not null
GROUP BY CRU, MILL
ORDER BY CRU, MILL DESC;

#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [ ]:
%%sql
SELECT NV, Number_of_producers
FROM (SELECT NV, count(NP) as Number_of_producers
      FROM production
      WHERE NV is not null
     GROUP BY NV)
WHERE Number_of_producers >3;

#### Exercise 2.6

Which producers have not produced any wine?

In [ ]:
%%sql
SELECT NP, NOM, PRENOM
FROM producer
WHERE NP is not NULL
EXCEPT 
SELECT production.NP, NOM, PRENOM
FROM producer, production
WHERE production.NP=producer.NP;

#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [ ]:
%%sql 
SELECT client.NB, NOM, PRENOM
FROM purchase, client, wine
WHERE MILL=1980 and client.NB=purchase.NB and wine.NV=purchase.NV
GROUP BY purchase.NB;

#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [ ]:
%%sql 
SELECT client.NB, NOM, PRENOM
FROM purchase, client, wine
WHERE client.NB is not null
EXCEPT 
SELECT client.NB, NOM, PRENOM
FROM purchase, client, wine
WHERE MILL=1980 and client.NB=purchase.NB and wine.NV=purchase.NV
GROUP BY purchase.NB;

#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [ ]:
%%sql
SELECT distinct purchase.NB, NOM, PRENOM
FROM wine, purchase, client
WHERE wine.NV=purchase.NV and client.NB = purchase.NB and MILL=1980
EXCEPT
SELECT distinct purchase.NB, NOM, PRENOM
FROM wine, purchase, client
WHERE wine.NV=purchase.NV and client.NB = purchase.NB and MILL!=1980;

#### Exercise 2.10

List all wines from 1980

In [ ]:
%%sql
SELECT NV, CRU, MILL, DEGRE
FROM wine
WHERE MILL=1980;

#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [ ]:
%%sql
SELECT wine.NV, CRU, MILL, DEGRE
FROM wine, purchase
WHERE wine.NV=purchase.NV and MILL=1980 and NB=2;

#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [ ]:
%%sql
SELECT distinct purchase.NB, NOM, PRENOM
FROM wine, purchase, client
WHERE wine.NV=purchase.NV and client.NB = purchase.NB and MILL=1980 
GROUP BY purchase.NB, NOM, PRENOM
HAVING count(purchase.NV)= (SELECT count(NV)
FROM wine
WHERE MILL=1980
GROUP BY MILL);